In [43]:
import requests
import pandas as pd
import numpy as np
from scipy.stats import linregress
from datetime import datetime, timedelta

In [45]:
# API활용한 데이터를 가져온 csv파일을 열어서 계산하기.

# 먼저 데이터 가져오기

krx_data = pd.read_csv('../API/krx_data.csv')
kospi_data = pd.read_csv('../API/kospi_1year_data.csv')
rf_data = pd.read_csv ('../API/rf.csv') # 국채 수익률


In [46]:
#시계열 자료로 변환

krx_data['BAS_DD'] = pd.to_datetime(krx_data['BAS_DD'], format='%Y%m%d', errors='coerce')
kospi_data['BAS_DD'] = pd.to_datetime(kospi_data['BAS_DD'], format = '%Y%m%d', errors='coerce')


In [49]:
kospi = kospi_data[kospi_data['IDX_NM'] == '코스피 200']
kospi

,BAS_DD,IDX_CLSS,IDX_NM,CLSPRC_IDX,CMPPREVDD_IDX,FLUC_RT,OPNPRC_IDX,HGPRC_IDX,LWPRC_IDX,ACC_TRDVOL,ACC_TRDVAL,MKTCAP
2,2023-10-23,KOSPI,코스피 200,315.50,-2.46,-0.77,317.43,318.30,315.09,89162326,4886900168746,1637491058487765
49,2023-10-24,KOSPI,코스피 200,318.43,2.93,0.93,317.34,318.63,311.77,93256078,5696794937037,1654982984217825
96,2023-10-25,KOSPI,코스피 200,316.49,-1.94,-0.61,319.26,319.59,316.30,91218212,5990647874872,1638326343330425
143,2023-10-26,KOSPI,코스피 200,307.75,-8.74,-2.76,311.36,312.37,307.75,110774427,7147843599760,1594021634219925
190,2023-10-27,KOSPI,코스피 200,308.53,0.78,0.25,309.68,310.43,307.24,92934386,5978754499550,1597031432203450
...,...,...,...,...,...,...,...,...,...,...,...,...
11235,2024-10-16,KOSPI,코스피 200,347.26,-3.81,-1.09,346.34,349.11,345.54,133287754,8490305831153,1874447502735365
11282,2024-10-17,KOSPI,코스피 200,347.73,0.47,0.14,348.84,349.18,346.32,146934034,8432270256638,1873192075557845
11329,2024-10-18,KOSPI,코스피 200,345.42,-2.31,-0.66,348.59,349.24,344.72,106414681,7200953817374,1862393453574040
11376,2024-10-21,KOSPI,코스피 200,346.62,1.20,0.35,346.25,348.62,345.12,103207235,6792100003923,1871020889812440


In [53]:
# 데이터 로드
kospi_200 = kospi_data[kospi_data['IDX_NM'] == '코스피 200'][['BAS_DD', 'FLUC_RT']]
kospi_200['Market_Return'] = pd.to_numeric(kospi_200['FLUC_RT'], errors='coerce') / 100  # 퍼센트에서 소수로 변환

krx_data['Stock_Return'] = pd.to_numeric(krx_data['FLUC_RT'], errors='coerce') / 100  # 퍼센트에서 소수로 변환

# 무위험 수익률 준비
rf_data['RF'] = pd.to_numeric(rf_data['CLSPRC_YD'], errors='coerce') / 100  # 퍼센트에서 소수로 변환

krx_data

,BAS_DD,ISU_CD,ISU_NM,MKT_NM,SECT_TP_NM,TDD_CLSPRC,CMPPREVDD_PRC,FLUC_RT,TDD_OPNPRC,TDD_HGPRC,TDD_LWPRC,ACC_TRDVOL,ACC_TRDVAL,MKTCAP,LIST_SHRS,Stock_Return
0,2023-10-23,095570,AJ네트웍스,KOSPI,-,3790,155,4.26,3640,3805,3595,134548,501411875,171507956610,45252759,0.0426
1,2023-10-23,006840,AK홀딩스,KOSPI,-,16730,-180,-1.06,16910,17000,16200,2974,49332920,221631695530,13247561,-0.0106
2,2023-10-23,027410,BGF,KOSPI,-,3210,-25,-0.77,3200,3265,3200,41660,134455890,307250899110,95716791,-0.0077
3,2023-10-23,282330,BGF리테일,KOSPI,-,130900,-800,-0.61,133800,133800,129400,41588,5433104300,2262463295400,17283906,-0.0061
4,2023-10-23,138930,BNK금융지주,KOSPI,-,7050,-100,-1.40,7220,7220,6980,1105950,7802828310,2270723487900,322088438,-0.0140
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232814,2024-10-22,079980,휴비스,KOSPI,-,2740,-60,-2.14,2810,2830,2700,33923,92704485,94530000000,34500000,-0.0214
232815,2024-10-22,005010,휴스틸,KOSPI,-,4000,-45,-1.11,4020,4055,3990,191517,767556150,224752300000,56188075,-0.0111
232816,2024-10-22,000540,흥국화재,KOSPI,-,3375,-15,-0.44,3390,3445,3365,18472,62402325,216818926875,64242645,-0.0044
232817,2024-10-22,000545,흥국화재우,KOSPI,-,5320,-40,-0.75,5280,5330,5190,719,3761390,4085760000,768000,-0.0075


In [57]:
kospi_200

,BAS_DD,FLUC_RT,Market_Return
2,2023-10-23,-0.77,-0.0077
49,2023-10-24,0.93,0.0093
96,2023-10-25,-0.61,-0.0061
143,2023-10-26,-2.76,-0.0276
190,2023-10-27,0.25,0.0025
...,...,...,...
11235,2024-10-16,-1.09,-0.0109
11282,2024-10-17,0.14,0.0014
11329,2024-10-18,-0.66,-0.0066
11376,2024-10-21,0.35,0.0035


In [59]:
print(merged_data.shape)  # 병합된 데이터프레임의 크기 확인
print(rf_data.shape)      # 무위험 수익률 데이터 크기 확인
print(rf_data['RF'])


(232819, 17)
(1, 2)
0    0.0296
Name: RF, dtype: float64


In [61]:
merged_data = merged_data.reset_index(drop=True)

# 데이터 병합 (시장 수익률과 개별 종목 데이터 병합)
merged_data = pd.merge(krx_data, kospi_200[['BAS_DD', 'Market_Return']], on='BAS_DD', how='inner')

# 2. 무위험 수익률 값 추출
rf_value = rf_data['RF'].iloc[0]  # 첫 번째 값을 추출

# 3. 병합된 데이터프레임에 무위험 수익률 추가
merged_data['RF'] = rf_value  # 모든 행에 동일한 값을 추가


# 결과 확인
print(merged_data.head())

      BAS_DD  ISU_CD   ISU_NM MKT_NM SECT_TP_NM  TDD_CLSPRC  CMPPREVDD_PRC  \
0 2023-10-23  095570   AJ네트웍스  KOSPI          -        3790            155   
1 2023-10-23  006840    AK홀딩스  KOSPI          -       16730           -180   
2 2023-10-23  027410      BGF  KOSPI          -        3210            -25   
3 2023-10-23  282330   BGF리테일  KOSPI          -      130900           -800   
4 2023-10-23  138930  BNK금융지주  KOSPI          -        7050           -100   

   FLUC_RT  TDD_OPNPRC  TDD_HGPRC  TDD_LWPRC  ACC_TRDVOL  ACC_TRDVAL  \
0     4.26        3640       3805       3595      134548   501411875   
1    -1.06       16910      17000      16200        2974    49332920   
2    -0.77        3200       3265       3200       41660   134455890   
3    -0.61      133800     133800     129400       41588  5433104300   
4    -1.40        7220       7220       6980     1105950  7802828310   

          MKTCAP  LIST_SHRS  Stock_Return  Market_Return      RF  
0   171507956610   45252759    

In [69]:
# 5. 베타 계산 함수
def calculate_beta(stock_returns, market_returns):
    """Calculate beta as covariance(stock, market) / variance(market)."""
    if len(stock_returns) > 1:  # 데이터가 충분한지 확인
        covariance = np.cov(stock_returns, market_returns)[0, 1]  # 공분산
        variance = np.var(market_returns)  # 시장의 분산
        return covariance / variance if variance != 0 else np.nan
    else:
        return np.nan

# 상위 200개 기업의 베타 계산
top_200_stocks = merged_data.sort_values(by='MKTCAP', ascending=False)['ISU_NM'].unique()[:200]  # 상위 200개 기업 선택
beta_results = []

for stock in top_200_stocks:
    stock_data = merged_data[merged_data['ISU_NM'] == stock]
    beta = calculate_beta(stock_data['Stock_Return'].dropna(), stock_data['Market_Return'].dropna())
    beta_results.append({'ISU_NM': stock, 'Beta': beta})

# 결과 데이터프레임 생성
beta_df = pd.DataFrame(beta_results)

# 결과 확인
print(beta_df.head())

# CSV 파일로 저장
beta_df.to_csv("top_200_betas.csv", index=False, encoding="utf-8-sig")


     ISU_NM      Beta
0      삼성전자  1.117357
1    SK하이닉스  1.647635
2  LG에너지솔루션  0.922005
3  삼성바이오로직스  0.472365
4       현대차  1.025313


In [71]:
beta_df

,ISU_NM,Beta
0,삼성전자,1.117357
1,SK하이닉스,1.647635
2,LG에너지솔루션,0.922005
3,삼성바이오로직스,0.472365
4,현대차,1.025313
...,...,...
195,동원시스템즈,1.017876
196,솔루엠,0.435295
197,코스모화학,1.536607
198,LX세미콘,0.760200
